### Import Libraries

In [4]:
import numpy as np
import pandas as pd
print(pd.__version__)
pd.options.display.max_colwidth = 500

0.20.3


## Import CSV's into pandas dataframes

In [6]:
df_submissions = pd.read_csv("reddit_submissions_wUTC_2018-05-10_1022.csv",
skipinitialspace=True,
header = 0,
sep = ',',
index_col=0,
encoding='latin1').fillna('')
df_submissions['post'] = 1
print(df_submissions.shape)
print(df_submissions.columns)

df_comments = pd.read_csv("reddit_comments_wUTC2018-05-10_1022.csv",
skipinitialspace=True,
header = 0,
sep = ',',
index_col=0,
encoding='latin1').fillna('')
df_comments['post'] = 1
print(df_comments.shape)
print(df_comments.columns)

(1362, 26)
Index(['id', 'subreddit_id', 'title', 'score', 'author', 'author_flair_text',
       'likes', 'archived', 'is_video', 'name', 'over_18', 'pinned',
       'quarantine', 'selftext', 'send_replies', 'num_comments',
       'num_crossposts', 'distinguished', 'created', 'created_utc',
       'comment_limit', 'approved_by', 'ups', 'downs', 'gilded', 'post'],
      dtype='object')
(6704, 20)
Index(['comment_id', '_submission', 'subreddit_id', 'score', 'author',
       'author_flair_text', 'archived', 'name', 'body', 'send_replies',
       'distinguished', 'created', 'created_utc', 'approved_by', 'link_id',
       'parent_id', 'ups', 'downs', 'gilded', 'post'],
      dtype='object')


## Renaming Columns

In [7]:
df_comments = df_comments.rename(columns={'_submission':'id',
                                         'body':'selftext'})

## Merge Dataframes 

In [8]:
df_all = df_submissions.append(df_comments)
df_all['comment'] = np.where(df_all['comment_id'].isnull(),0,1)
df_all['submission'] = np.where(df_all['comment']==0,1,0)
print(df_all.shape)
df_all.columns

(8066, 31)


Index(['approved_by', 'archived', 'author', 'author_flair_text', 'comment_id',
       'comment_limit', 'created', 'created_utc', 'distinguished', 'downs',
       'gilded', 'id', 'is_video', 'likes', 'link_id', 'name', 'num_comments',
       'num_crossposts', 'over_18', 'parent_id', 'pinned', 'post',
       'quarantine', 'score', 'selftext', 'send_replies', 'subreddit_id',
       'title', 'ups', 'comment', 'submission'],
      dtype='object')

## Format variables related to time

The date of Reddit's posts are stored in seconds <br>
* the initial date is '1970-01-01 00:00:00' <br>
* The unit argument stands for the unit of time measure : 's'
* Split the date and the time in separeted variables
* The variable 'created' is reffering to the Timestamp from ? ... We can convert it to UTC by removing 8 hours

In [9]:
start = 0
start = pd.to_datetime(start, unit='s')
print(start)

1970-01-01 00:00:00


### Compare created & created_UTC

### Format Date & Time

In [10]:
df_all['datetime'] = pd.to_datetime(df_all['created'], unit='s')
df_all['datetime'] = pd.DatetimeIndex(df_all['datetime'])
print(df_all['datetime'][0:2])

2   2018-05-04 00:44:18
4   2018-05-04 01:02:20
Name: datetime, dtype: datetime64[ns]


In [97]:
temp = pd.to_datetime(df_all['created'], unit='s')
temp = pd.DatetimeIndex(temp)
print(temp[0:2])

DatetimeIndex(['2018-04-23 01:08:24', '2018-04-22 20:22:23'], dtype='datetime64[ns]', name='created', freq=None)


In [98]:
df_all['date'] = temp.date
df_all['time'] = temp.time
print(df_all['date'][0:2])
print(df_all['time'][0:2])

0    2018-04-23
1    2018-04-22
Name: date, dtype: object
0    01:08:24
1    20:22:23
Name: time, dtype: object


## Time series Analysis

In [99]:
from pandas import Series, DataFrame, Panel, Grouper

In [100]:
df_all = df_all.set_index('datetime')

In [105]:
sub_permonth = df_all['submission'].resample("M").sum()
print(sub_permonth)

comment_permonth = df_all['comment'].resample("M").sum()
print(comment_permonth)

datetime
2017-11-30    189
2017-12-31    259
2018-01-31    327
2018-02-28    374
2018-03-31    402
2018-04-30    363
2018-05-31     18
Freq: M, Name: submission, dtype: int32
datetime
2017-11-30     843
2017-12-31    1188
2018-01-31    1480
2018-02-28    1491
2018-03-31    1903
2018-04-30    1962
2018-05-31     138
Freq: M, Name: comment, dtype: int32
